<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/Preprocessing_Pipeline_for_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter
nlp = spacy.load('en')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [ ]:
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)

In [ ]:
answer_start=[]
answer_end=[]

for i in range(0,20000):
  text=str(data['context_translated'][i])
  answer=str(data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [ ]:
data['answer_start']=answer_start
data['answer_end']=answer_end
data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව,-1,14
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි,-1,44
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්,-1,14
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
...,...,...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය,-1,13
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා,-1,10
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය,312,320


In [ ]:
df = data[data.answer_start != -1]
df

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...
19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [ ]:
df=df.reset_index()
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('indexed_data.csv')
!cp indexed_data.csv "drive/My Drive/"

In [ ]:
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [1]:
from google.colab import files
uploaded = files.upload()

Saving indexed_data.csv to indexed_data.csv


In [2]:

import pandas as pd


In [3]:
df=pd.read_csv("indexed_data.csv")

In [4]:

df

,Unnamed: 0,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...,...
8993,8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


# **Importing libraries packages necessary for data pre processing pipeline**

In [15]:
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk import wordnet
from nltk.tokenize import word_tokenize
import string


import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter
nlp = spacy.load('en')

In [7]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Build a list of dictionaries with ( context , question , label ) as keys**

In [ ]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list

# **Function to pre-process paragraph of text**



In [8]:
stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        #Replacing special worsds
        #document = re.sub(r'නැකඩ්', " ", document)
        #document = re.sub(r'අඩීචි'," ", document)
        #document = re.sub(r'සොංටැං'," ",document)
        #document = re.sub(r'``', " ", document)
        #document = re.sub(r'[^\w\s]','',document)
        #document = re.sub(r'\_','',document)


        # Converting to Lowercase
        document = document.lower()

        #removing digits from the corpus since FastText unable to embedd digits
        #document= re.sub(r'\b\d+\b', '', document)
        

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

# **Function to create list of Context, Question and Context+Question lists**

In [35]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['text_translated']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  table1 = str.maketrans('', '', string.punctuation)
  stripped_context = [w.translate(table1) for w in context]

  table2 = str.maketrans('', '', string.punctuation)
  stripped_question = [w.translate(table2) for w in question]

  table3 = str.maketrans('', '', string.punctuation)
  stripped_answer = [w.translate(table2) for w in answer]

  lis=stripped_context+stripped_question
  return lis,stripped_context,stripped_question,stripped_answer



# **Function to build vocabulary of words , word to index and index to word mapping**

In [36]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=word_tokenize(sentences)
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=word_tokenize(line)
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab
   


In [26]:
lis,stripped_context,stripped_question=gather_text_for_vocab(df)
word2idx, idx2word, word_vocab=build_word_vocab(lis)

In [12]:
print(len(word2idx))
print(len(idx2word))
print(len(word_vocab))

59267
59267
17996


# **Function to create character vocabulary and character to index mapping** 

In [16]:
def build_char_vocab(vocab_text):

    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'<unk>')
    char_vocab.insert(1,'<pad>')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

In [18]:
char2idx, char_vocab=build_char_vocab(lis)
print(len(char2idx))
print(len(char_vocab))

raw-char-vocab: 406
char-vocab-intersect: 122
char2idx-length: 124
124
124


# **Function to create id's for pre-processed context words**

In [19]:
def context_to_ids(text, word2idx):
   

    context_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

In [23]:
text=lis[0]
context_to_ids(text, word2idx)

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 25,
 26,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 4,
 45,
 46,
 41,
 47,
 12,
 42,
 48,
 49,
 50,
 51,
 12,
 52,
 53,
 54,
 55,
 56,
 14,
 57,
 58,
 59,
 55,
 60,
 61,
 51,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 31,
 69,
 70,
 11,
 71,
 72,
 73,
 74,
 75,
 76,
 77]

# **Function to create id's for pre-processed questions**

In [24]:
def question_to_ids(text, word2idx):
 

    question_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [28]:
q_text=stripped_question[0]
print(q_text)
question_to_ids(q_text, word2idx)

බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද


[3189, 229, 597, 7350, 3110, 57425]

# **Creating new pre processed and labeled dataset ( Cleaned dataset )**

In [31]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [33]:
full=pd.read_csv("english_sinhala_combined.csv")
full=full.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)
full

,context_translated,question_translated,text_translated
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්
...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ..."
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය


In [37]:
lis,stripped_context,stripped_question,stripped_answer=gather_text_for_vocab(full)

In [38]:
cleaned_data=pd.DataFrame(list(zip(stripped_context,stripped_question,stripped_answer)), columns =['context_translated', 'question_translated','text_translated'])
cleaned_data

,context_translated,question_translated,text_translated
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්
1,2002 රදර්ශනය ආරම්භ සිට අයිඩල් ජයග් රාහකයින් දහ...,ඇමරිකානු අයිඩල් තරඟකරු රිස් ඩොට් කුමන රාන්තයෙන්ද,උතුරු කැරොලිනාව
2,ශතවර්ෂයේ භාගයේ ජර්මානු ගෝත් රිකයන් රෝමානු රදේශ...,සියවසේ ඉදිකරන,බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන
3,බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ඇල්බමයේ ...,දශකයේ කැනේ විසින් සාදන බීට්ස් කවුද,දේශීය කලාකරුවන්
4,ටුවාලුආන් භාෂාව ඉංග් රීසි ටුවාලු ජාතික භාෂා ටු...,ඉංග් රීසි හැරුණු ටුවාලු ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්
...,...,...,...
19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් රජාතන්ත් රවාදයට රාජ් යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය රකාශ කිරීමට campaign ෝෂ...
19996,පැරණි ඉංග් රීසි භාෂාවෙන් ලතින් භාෂාවෙන් නිශ්චි...,ලතින් හෝඩිය හඳුන්වාදීමට පැරණි ඉංග් රීසි ලිවූයේ...,රූනික් පද්ධතිය
19997,රූනික් පද්ධතිය ඇරිස්ටෝෆනේස්ගේ නාට් හාස් පැරණිත...,ප්ලේටෝ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත් යයක්ද,දාර්ශනික
19998,ආරම්භක අවසන් දිනයන් වශයෙන් දකුණු අර්ධගෝලයේ රති...,දකුණු අර්ධගෝලයේ කුමන පරිච්ඡේදයේදී චිලී රධාන භූ...,ගිම්හානය


In [39]:
answer_start=[]
answer_end=[]

for i in range(0,20000):
  text=str(cleaned_data['context_translated'][i])
  answer=str(cleaned_data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [40]:
cleaned_data['answer_start']=answer_start
cleaned_data['answer_end']=answer_end
cleaned_data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,2002 රදර්ශනය ආරම්භ සිට අයිඩල් ජයග් රාහකයින් දහ...,ඇමරිකානු අයිඩල් තරඟකරු රිස් ඩොට් කුමන රාන්තයෙන්ද,උතුරු කැරොලිනාව,-1,14
2,ශතවර්ෂයේ භාගයේ ජර්මානු ගෝත් රිකයන් රෝමානු රදේශ...,සියවසේ ඉදිකරන,බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන,-1,36
3,බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ඇල්බමයේ ...,දශකයේ කැනේ විසින් සාදන බීට්ස් කවුද,දේශීය කලාකරුවන්,-1,14
4,ටුවාලුආන් භාෂාව ඉංග් රීසි ටුවාලු ජාතික භාෂා ටු...,ඉංග් රීසි හැරුණු ටුවාලු ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,44,54
...,...,...,...,...,...
19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් රජාතන්ත් රවාදයට රාජ් යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය රකාශ කිරීමට campaign ෝෂ...,27,88
19996,පැරණි ඉංග් රීසි භාෂාවෙන් ලතින් භාෂාවෙන් නිශ්චි...,ලතින් හෝඩිය හඳුන්වාදීමට පැරණි ඉංග් රීසි ලිවූයේ...,රූනික් පද්ධතිය,-1,13
19997,රූනික් පද්ධතිය ඇරිස්ටෝෆනේස්ගේ නාට් හාස් පැරණිත...,ප්ලේටෝ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත් යයක්ද,දාර්ශනික,-1,7
19998,ආරම්භක අවසන් දිනයන් වශයෙන් දකුණු අර්ධගෝලයේ රති...,දකුණු අර්ධගෝලයේ කුමන පරිච්ඡේදයේදී චිලී රධාන භූ...,ගිම්හානය,266,274


In [45]:
new_df = cleaned_data[cleaned_data.answer_start != -1]
new_df=new_df.reset_index()



,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව ඉංග් රීසි ටුවාලු ජාතික භාෂා ටු...,ඉංග් රීසි හැරුණු ටුවාලු ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,44,54
2,8,ජේන් ඔස්ටන් රධාන තදාසන්න රදේශ නිරිත දෙසින් මිඩ...,රිච්මන්ඩ් නිරිතදිග තදාසන්න රදේශය කුමක්ද,මිඩ්ලෝතියන්,43,54
3,9,දෙවන යුද්ධයෙන් ජර්මනිය මිත් පාක්ෂිකයන් විසින් ...,දෙවන යුද්ධයෙන් ජර්මනියේ මුලින් රාන්ත කීයක් පැව...,,0,0
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී රංශයේ අවනීතිය යාක...,දහස් ගණනකට යුද්ධය මරණය පිළිබඳ අපේක්ෂාවෙන් නැපෝ...,ඔහුගේ විවේචකයෝ,135,149
...,...,...,...,...,...,...
10909,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් යූඑස්බී සම්බන්ධතාවයට ...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,267,272
10910,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට කින...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් ස්වාභාවික රිද්...,සර්කැඩියානු රිද්මය,120,138
10911,19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් රජාතන්ත් රවාදයට රාජ් යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය රකාශ කිරීමට campaign ෝෂ...,27,88
10912,19998,ආරම්භක අවසන් දිනයන් වශයෙන් දකුණු අර්ධගෝලයේ රති...,දකුණු අර්ධගෝලයේ කුමන පරිච්ඡේදයේදී චිලී රධාන භූ...,ගිම්හානය,266,274


In [46]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [47]:
new_df.to_csv('cleaned_data.csv')
!cp cleaned_data.csv "drive/My Drive/"